# Information Retrival
## Assignment 2, Group 42
### Question 1
#### Team Members: Harsh Bandhey, Md Talib, Arshdeep Singh

------------------------------------------

In [1]:
import os
import re
import glob
import nltk
import pickle
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from collections import OrderedDict

In [2]:
# nltk.download('wordnet')
# nltk.download('stopwords')

In [3]:
files = list()
for dirpath, dirs, fls in os.walk('stories'):  
    for filename in fls: 
        path = os.path.join(dirpath,filename)
        fname = os.path.join('',filename)
        if not fname.startswith(".") and not fname.endswith("html"):
            files.append(path)
print(len(files))

467


In [4]:
stop_words = set(stopwords.words('english'))
def preprocess(text):        
    text = text.lower() #1
    text = re.sub(r'[^\w\s]', ' ', text) #4
    tokens = word_tokenize(text) #2
    tokens = [word for word in tokens if word not in stop_words] #3
    tokens = [token for token in tokens if token.strip()!=''] #5
#     tokens = [re.sub(r'[^a-zA-Z0-9\s]','',text) for token in tokens] #Extrastep
    return tokens

In [5]:
def createPositionalIndex(files):
    index = {}
    ps = PorterStemmer()
    for i in tqdm(range(len(files))):
        with open(files[i], encoding= 'unicode-escape') as f:
            doc = [a for a in preprocess(f.read())]
        for idx, word in enumerate(doc):
            stemmed = word
            stemmed = ps.stem(word)
            if not stemmed in index:
                index[stemmed] = dict()
                index[stemmed][i] = [idx,]
            else: 
                if i in index[stemmed].keys():
                    index[stemmed][i].append(idx)
                else:
                    index[stemmed][i] = [idx,]
    return index

In [6]:
pi = createPositionalIndex(files)
with open("posindex.pkl","wb") as f:
    pickle.dump(pi,f)

100%|████████████████████████████████████████████████████████████████████████████████| 467/467 [00:37<00:00, 12.54it/s]


In [7]:
with open("posindex.pkl","rb") as f:
    pi = pickle.load(f)

In [8]:
def searchDocs(index,query):
    ps = PorterStemmer()
    query = [a for a in preprocess(query)]
#     query = [ps.stem(word) for word in query]
    
    print("Processed Query: ",query)
    if len(query)==1:
        return index[query[0]].keys()
    
    init = list()    
    init_word = query[0]
        
    for doc,positions in index[init_word].items():
        for position in positions:
            init.append((doc, position))
            
    other_query_words = query[1:]
            
    matched_docs = []
    
    for pair in init:
        init_doc = pair[0]
        init_pos = pair[1]
        count = 0
        for word in other_query_words:
            next_pos = init_pos+1
            docs_list = index[word].keys()
            if init_doc in docs_list:
                doc_positions = index[word].get(init_doc,dict())
#                 print(doc,word,init_pos,next_pos,doc_positions)
                if next_pos in doc_positions:
                    count += 1
                else:
                    count += 1
                    break

            if count == len(other_query_words):
                matched_docs.append(pair[0])
               
    matched_docs = set(matched_docs)
    print("Found ",len(matched_docs)," documents")
    print()
    print("Printing all matched documents")
    for i in matched_docs:
        print(files[i])
    return matched_docs

In [9]:
found = searchDocs(pi,"Good-day")

Processed Query:  ['good', 'day']
Found  21  documents

Printing all matched documents
stories\13chil.txt
stories\startrek.txt
stories\superg1
stories\mazarin.txt
stories\melissa.txt
stories\enchdup.hum
stories\aesop11.txt
stories\aesopa10.txt
stories\fantasy.hum
stories\fantasy.txt
stories\fic5
stories\outcast.dos
stories\forgotte
stories\brain.damage
stories\breaks2.asc
stories\bruce-p.txt
stories\SRE\srex.txt
stories\history5.txt
stories\horswolf.txt
stories\hound-b.txt
stories\sick-kid.txt


In [10]:
query = input("Query")
found = searchDocs(pi,query)

Querygood day
Processed Query:  ['good', 'day']
Found  21  documents

Printing all matched documents
stories\13chil.txt
stories\startrek.txt
stories\superg1
stories\mazarin.txt
stories\melissa.txt
stories\enchdup.hum
stories\aesop11.txt
stories\aesopa10.txt
stories\fantasy.hum
stories\fantasy.txt
stories\fic5
stories\outcast.dos
stories\forgotte
stories\brain.damage
stories\breaks2.asc
stories\bruce-p.txt
stories\SRE\srex.txt
stories\history5.txt
stories\horswolf.txt
stories\hound-b.txt
stories\sick-kid.txt
